# LlamaIndex Bottoms-Up Development - LLMs and Prompts
This notebook walks through testing an LLM using the primary prompt templates used in llama-index.

In [ ]:
import openai
import os

openai.api_key = "sk-svcacct-123456"
os.environ["OPENAI_API_KEY"] = "sk-svcacct-123456"

## Setup
In this section, we load a test document, create an LLM, and copy prompts from llama-index to test with.

First, let's load a quick document to test with. Right now, we will just load it as plain text, but we can do other operations later!

In [3]:
with open("../docs/getting_started/starter_example.md", "r") as f:
    text = f.read()

Next, we create our LLM!

In [4]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o-mini", temperature=0)

LlamaIndex uses some simple templates under the hood for answering queries -- mainly a `text_qa_template` for obtaining initial answers, and a `refine_template` for refining an existing answer when all the text does not fit into one LLM call.

Let's copy the default templates, and test out our LLM with a few questions.

In [5]:
from llama_index.core.prompts.base import PromptTemplate

text_qa_template = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = PromptTemplate(
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

Now, lets test a few questions!

## Text QA Template Testing

In [6]:
question = "How can I uninstall llama-index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To uninstall LlamaIndex, you can use the package manager that you used to install it. If you installed it using `pip`, you can uninstall it by running the following command in your terminal:

```bash
pip uninstall llama-index
```

If you used a different package manager, such as `conda`, you would use the corresponding command for that manager. For example, with `conda`, you would run:

```bash
conda remove llama-index
```

Make sure to check your environment and the package manager you used for installation to ensure you are using the correct command.


In [7]:
question = "How do I create an index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To create an index using LlamaIndex, follow these steps:

1. **Load the Documents**: Use the `SimpleDirectoryReader` to load the documents from a specified directory (in this case, the `data` folder).

   ```python
   from llama_index import VectorStoreIndex, SimpleDirectoryReader

   documents = SimpleDirectoryReader('data').load_data()
   ```

2. **Build the Index**: Create an index from the loaded documents using `VectorStoreIndex`.

   ```python
   index = VectorStoreIndex.from_documents(documents)
   ```

After executing these steps, you will have successfully created an index over the documents in the specified directory. You can then use this index for querying.


In [8]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```

## Refine Template Testing

In [9]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = refine_template.format(context_msg=text, query_str=question, existing_answer=existing_answer)
response = llm.complete(prompt)
print(response.text)

To create an index using LlamaIndex, follow these steps in your Python code:

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Load documents from the 'data' folder
documents = SimpleDirectoryReader('data').load_data()

# Build the index from the loaded documents
index = VectorStoreIndex.from_documents(documents)

# To persist the index to disk
index.storage_context.persist()

# To reload the index from disk
from llama_index import StorageContext, load_index_from_storage

# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")

# Load index
index = load_index_from_storage(storage_context)
```


### Chat Example
The LLM also has a `chat` method that takes in a list of messages, to simulate a chat session. 

In [10]:
from llama_index.core.base.llms.types import ChatMessage

chat_history = [
    ChatMessage(role="system", content="You are a helpful QA chatbot that can answer questions about llama-index."),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: To create an index using LlamaIndex, you typically follow these steps:

1. **Install LlamaIndex**: If you haven't already, make sure to install the LlamaIndex library. You can do this using pip:

   ```bash
   pip install llama-index
   ```

2. **Import Required Modules**: In your Python script or notebook, import the necessary modules from LlamaIndex.

   ```python
   from llama_index import Document, GPTSimpleVectorIndex
   ```

3. **Prepare Your Documents**: Create a list of documents that you want to index. Each document can be represented as a `Document` object.

   ```python
   documents = [
       Document("This is the first document."),
       Document("This is the second document."),
       Document("This is the third document.")
   ]
   ```

4. **Create the Index**: Use the `GPTSimpleVectorIndex` (or another index type depending on your needs) to create an index from your documents.

   ```python
   index = GPTSimpleVectorIndex(documents)
   ```

5. **Query the Ind

## Conclusion

In this notebook, we covered the low-level LLM API, and tested out some basic prompts with out documentation data.